In [1]:
from environment import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
auc = pd.read_table('/home/k/Downloads/v22.data.auc_sensitivities.txt')
print(auc.shape)

cpd = pd.read_table('/home/k/Downloads/v22.meta.per_compound.txt', index_col=0)
print(cpd.shape)

ccl = pd.read_table('/home/k/Downloads/v22.meta.per_cell_line.txt', index_col=0)
print(ccl.shape)

(260496, 3)
(481, 9)
(664, 8)


In [3]:
compound_x_cellline = pd.DataFrame(
    index=sorted(set(cpd['cpd_name'])),
    columns=sorted(set(ccl['ccl_name']))
)
print(compound_x_cellline.shape)

cpd_d = cpd['cpd_name'].to_dict()
ccl_d = ccl['ccl_name'].to_dict()

for i, (i_cpd, i_ccl, a) in auc.iterrows():
    
    # Get names
    cpd_n = cpd_d[i_cpd]
    ccl_n = ccl_d[i_ccl]
   
    # Get current AUC
    a_ =  compound_x_cellline.loc[cpd_n, ccl_n]
    
    # If the current AUC is not set, set with this AUC
    if pd.isnull(a_):
        compound_x_cellline.loc[cpd_n, ccl_n] = a
        
    # If this AUC is smaller than the current AUC, set with this AUC
    elif a < a_:
            print()
            print(cpd_n, ccl_n, a_, a)
            compound_x_cellline.loc[cpd_n, ccl_n] = a

(481, 645)

ML311 CCFSTTG1 15.0 14.352

ML311 NCIH460 13.004 12.245

ML311 IGROV1 13.147 12.092

ML311 OAW28 13.205 11.837

ML311 ASPC1 13.483 13.2

ML311 CAL51 12.028 11.655

ML311 NCIH1869 14.149 13.606

ML311 LOXIMVI 13.386 12.332

ML311 MKN74 13.969 12.234

zebularine CCFSTTG1 14.999 14.634

zebularine A549 14.507 13.312

zebularine NCIH460 14.581 12.742

zebularine ASPC1 15.699 13.944

zebularine NCIH1869 14.726 14.633

zebularine MKN74 13.732 13.065

BRD-A02303741 A549 15.0 14.915

BRD-A02303741 OAW28 16.239 15.193

BRD-A02303741 CAL51 15.627 15.0

BRD-A02303741 NCIH1869 15.0 14.1

BRD-A02303741 AGS 16.331 14.669

BRD-A02303741 KE39 16.132 14.815

BRD-A02303741 MKN74 15.464 15.0

importazole NCIH1869 14.647 14.014

elocalcitol CCFSTTG1 16.144 15.844

elocalcitol A549 14.975 14.419

elocalcitol OAW28 17.306 14.03

elocalcitol SUIT2 15.566 14.905

elocalcitol A375 14.053 12.56

elocalcitol NCIH1869 16.405 14.638

elocalcitol KE39 13.735 13.709

elocalcitol MKN74 16.042 14.189

eloc

YK 4-279 A375 13.495 12.587

YK 4-279 KE39 14.121 13.745

YK 4-279 MKN74 13.669 13.368

fumonisin B1 CCFSTTG1 14.842 14.832

fumonisin B1 A549 15.0 14.071

fumonisin B1 NCIH460 15.0 14.992

fumonisin B1 NCIH520 15.273 15.0

fumonisin B1 IGROV1 15.323 14.105

fumonisin B1 OAW28 15.341 14.229

fumonisin B1 CAL51 14.954 14.811

fumonisin B1 NCIH1869 15.115 14.356

fumonisin B1 DU145 15.598 15.294

fumonisin B1 KE39 14.723 14.408

fumonisin B1 SKUT1 15.528 14.91

ifosfamide A549 15.422 12.264

ifosfamide NCIH1299 15.0 14.654

ifosfamide CAL51 15.0 14.892

ifosfamide A375 15.967 13.935

salermide CCFSTTG1 15.374 15.335

salermide OAW28 14.886 14.783

salermide NCIH1869 15.504 13.104

salermide SKUT1 14.334 14.241

BCL-LZH-4 ASPC1 15.677 14.842

BCL-LZH-4 NCIH1869 14.442 13.497

isoevodiamine CCFSTTG1 13.199 13.145

isoevodiamine A549 12.733 12.686

isoevodiamine NCIH460 14.077 13.626

isoevodiamine OAW28 12.974 12.012

isoevodiamine SUIT2 13.434 12.532

isoevodiamine A375 13.664 12.617

iso

BMS-270394 A549 15.583 15.347

BMS-270394 A549 15.347 14.998

BMS-270394 NCIH460 16.365 15.396

BMS-270394 IGROV1 16.646 14.812

BMS-270394 ASPC1 15.359 14.107

BMS-270394 SUIT2 15.357 15.0

BMS-270394 A375 14.901 14.626

BMS-270394 LOXIMVI 14.663 13.764

BMS-270394 AGS 15.959 14.995

GSK-3 inhibitor IX CCFSTTG1 13.992 13.465

GSK-3 inhibitor IX NCIH1299 15.405 13.491

GSK-3 inhibitor IX NCIH460 15.774 13.994

GSK-3 inhibitor IX IGROV1 12.429 11.88

GSK-3 inhibitor IX CAL51 14.261 12.93

GSK-3 inhibitor IX A375 12.741 12.091

GSK-3 inhibitor IX NCIH1869 11.856 11.554

CIL41 IGROV1 13.228 12.999

triazolothiadiazine CCFSTTG1 11.247 10.63

triazolothiadiazine A549 10.108 9.5688

triazolothiadiazine NCIH460 12.89 11.075

triazolothiadiazine IGROV1 10.717 8.475

triazolothiadiazine OAW28 12.283 10.263

triazolothiadiazine ASPC1 12.919 12.324

triazolothiadiazine CAL51 7.9222 7.4228

triazolothiadiazine LOXIMVI 9.0293 8.1338

triazolothiadiazine KE39 14.463 13.656

triazolothiadiazine MKN74


BRD-K13999467 CCFSTTG1 14.764 14.613

BRD-K13999467 A549 14.955 13.643

BRD-K13999467 A549 13.643 13.558

BRD-K13999467 IGROV1 15.0 14.23

BRD-K13999467 OAW28 14.567 12.934

BRD-K13999467 SUIT2 15.156 13.949

BRD-K13999467 KE39 15.0 14.093

BRD-K13999467 MKN74 14.592 14.278

BRD-K13999467 SKUT1 14.614 14.576

BRD-K14844214 A549 14.932 14.802

BRD-K14844214 ASPC1 14.199 14.005

BRD-K14844214 CAL51 14.917 14.84

BRD-K14844214 LOXIMVI 14.195 13.279

BRD-K14844214 DU145 14.82 14.459

R428 CCFSTTG1 15.029 12.009

R428 A549 12.433 11.936

R428 IGROV1 12.611 11.868

R428 OAW28 12.185 12.039

R428 SKUT1 11.473 11.255

gemcitabine CCFSTTG1 13.918 13.509

gemcitabine CAL51 7.693 6.2877

gemcitabine NCIH1869 13.129 11.829

gemcitabine KE39 12.921 12.676

gemcitabine MKN74 13.973 12.173

gemcitabine SKUT1 8.8338 8.6876

NVP-ADW742 A549 12.065 10.035

NVP-ADW742 IGROV1 13.365 11.668

NVP-ADW742 OAW28 12.58 10.979

NVP-ADW742 ASPC1 12.21 11.757

NVP-ADW742 NCIH1869 13.617 12.905

NVP-ADW742 LOXIMVI

STF-31 ASPC1 11.345 11.159

STF-31 SUIT2 16.461 15.775

STF-31 A375 12.758 10.175

STF-31 LOXIMVI 12.468 11.301

STF-31 DU145 10.582 10.156

STF-31 MKN74 11.762 11.481

BRD8899 CCFSTTG1 14.615 13.933

BRD8899 A549 15.0 14.04

BRD8899 NCIH460 15.235 15.102

BRD8899 IGROV1 15.0 14.562

BRD8899 ASPC1 14.741 14.209

BRD8899 CAL51 15.858 14.931

CR-1-31B CCFSTTG1 9.6797 7.8827

CR-1-31B A549 8.0794 7.5824

CR-1-31B NCIH460 11.246 10.306

CR-1-31B IGROV1 6.8205 6.532

CR-1-31B A375 8.3674 5.7314

CR-1-31B KE39 10.317 9.315

CR-1-31B SKUT1 9.1474 7.9938

1S,3R-RSL-3 CCFSTTG1 10.675 10.527

1S,3R-RSL-3 NCIH460 14.746 14.022

1S,3R-RSL-3 NCIH520 11.586 11.141

1S,3R-RSL-3 IGROV1 5.9223 5.685

1S,3R-RSL-3 OAW28 8.0271 2.7792

1S,3R-RSL-3 A375 7.245 6.7914

1S,3R-RSL-3 NCIH1869 9.3437 7.4352

1S,3R-RSL-3 LOXIMVI 3.6667 0.43721

sorafenib IGROV1 12.961 11.733

sorafenib ASPC1 12.683 12.46

sorafenib CAL51 13.871 13.154

sorafenib NCIH1869 13.915 13.225

sorafenib LOXIMVI 12.476 11.866

sorafenib D

...


ML029 IGROV1 14.582 14.504

ML029 OAW28 14.383 14.293

ML029 NCIH1869 14.827 14.569

ML029 LOXIMVI 14.819 14.783

ML029 KE39 13.869 12.778

ML029 MKN74 14.116 13.795

serdemetan CCFSTTG1 13.897 13.597

serdemetan A549 14.317 13.812

serdemetan NCIH460 14.688 13.386

serdemetan NCIH1869 14.736 14.607

MG-132 NCIH1869 11.154 10.733

MG-132 SKUT1 11.918 11.563

PYR-41 CCFSTTG1 14.813 14.583

PYR-41 A549 15.072 14.958

PYR-41 IGROV1 14.65 14.404

PYR-41 CAL51 15.873 14.798

CAL-101 A549 14.645 12.938

CAL-101 IGROV1 13.143 12.414

CAL-101 OAW28 13.298 12.61

CAL-101 CAL51 13.696 13.658

CAL-101 NCIH1869 13.722 13.213

CAL-101 MKN74 13.491 13.378

Mdivi-1 CCFSTTG1 17.795 16.041

Mdivi-1 A549 14.566 13.539

Mdivi-1 NCIH520 14.678 14.552

Mdivi-1 IGROV1 14.396 14.336

Mdivi-1 OAW28 15.328 12.747

Mdivi-1 ASPC1 14.85 14.682

Mdivi-1 CAL51 14.363 14.285

Mdivi-1 NCIH1869 15.902 13.935

Mdivi-1 LOXIMVI 16.019 14.402

Mdivi-1 MKN74 15.75 14.291

BRD-K61166597 CCFSTTG1 15.784 15.569

BRD-K6116659


pifithrin-alpha A549 13.709 12.942

pifithrin-alpha NCIH460 15.22 13.762

pifithrin-alpha SUIT2 15.428 11.396

pifithrin-alpha CAL51 14.646 13.58

pifithrin-alpha A375 14.54 13.562

pifithrin-alpha NCIH1869 15.357 14.317

Compound 23 citrate A549 14.197 13.292

Compound 23 citrate IGROV1 13.139 12.505

Compound 23 citrate OAW28 13.075 12.161

Compound 23 citrate A375 14.493 14.143

Compound 23 citrate LOXIMVI 13.967 9.6617

Compound 23 citrate KE39 12.905 12.849

Compound 23 citrate MKN74 14.267 12.444

SB-431542 A549 14.709 14.217

SB-431542 IGROV1 15.296 14.002

SB-431542 SUIT2 14.498 14.347

SB-431542 A375 15.426 14.931

SB-431542 LOXIMVI 15.409 14.577

SB-431542 SKUT1 14.172 13.06

KU-0063794 A549 12.584 12.236

KU-0063794 NCIH460 14.613 14.266

KU-0063794 IGROV1 9.9973 9.5942

KU-0063794 OAW28 13.632 11.577

GDC-0879 A549 14.903 14.808

GDC-0879 A549 14.808 14.703

GDC-0879 NCIH520 14.907 14.568

GDC-0879 IGROV1 14.971 14.429

GDC-0879 OAW28 18.047 15.148

GDC-0879 ASPC1 14.283 1

JW-74 A549 13.88 13.024

JW-74 IGROV1 14.374 13.889

JW-74 OAW28 13.916 13.413

JW-74 NCIH1869 14.66 14.351

JW-74 MKN74 14.059 13.684

omacetaxine mepesuccinate CCFSTTG1 8.0504 7.2033

omacetaxine mepesuccinate IGROV1 7.9928 6.9599

omacetaxine mepesuccinate LOXIMVI 4.5163 4.307

YM-155 CCFSTTG1 13.458 12.854

YM-155 OAW28 11.017 10.549

YM-155 ASPC1 10.619 10.05

YM-155 CAL51 9.6855 8.9667

YM-155 A375 11.185 11.068

YM-155 NCIH1869 8.4133 8.1576

YM-155 KE39 11.139 11.09

A-804598 CCFSTTG1 14.726 14.623

A-804598 ASPC1 14.541 14.212

A-804598 LOXIMVI 15.034 13.931

EX-527 A549 15.0 12.81

EX-527 NCIH460 14.722 12.914

EX-527 ASPC1 15.348 14.42

EX-527 CAL51 14.916 12.575

EX-527 A375 15.0 14.999

EX-527 SKUT1 13.953 13.734

vandetanib CCFSTTG1 13.988 13.63

vandetanib A549 13.165 12.521

vandetanib A549 12.521 12.283

vandetanib NCIH460 14.106 13.811

vandetanib IGROV1 10.983 10.42

vandetanib ASPC1 12.836 12.768

vandetanib A375 13.523 13.457

vandetanib NCIH1869 13.068 12.925

van

sirolimus A549 14.377 14.362

sirolimus A549 14.362 11.494

sirolimus NCIH520 7.2744 6.9646

sirolimus IGROV1 10.162 10.026

sirolimus OAW28 12.238 9.4242

sirolimus CAL51 9.2433 8.0333

sirolimus LOXIMVI 10.719 10.589

spautin-1 CCFSTTG1 14.484 13.819

spautin-1 A549 14.618 13.04

spautin-1 IGROV1 14.063 13.918

spautin-1 ASPC1 13.899 13.671

spautin-1 SUIT2 15.488 13.326

spautin-1 DU145 12.928 11.885

spautin-1 SKUT1 11.517 11.138

BRD-K85133207 A549 14.531 14.507

BRD-K85133207 A549 14.507 14.232

BRD-K85133207 OAW28 14.223 13.835

BRD-K85133207 SUIT2 14.886 14.768

BRD-K85133207 CAL51 14.808 14.606

BRD-K85133207 AGS 15.902 14.202

neratinib CCFSTTG1 14.613 14.275

neratinib A549 13.435 12.922

neratinib A549 12.922 11.933

neratinib OAW28 13.017 10.723

neratinib LOXIMVI 12.852 11.344

neratinib SKUT1 13.514 13.122

AZD7762 CCFSTTG1 9.7433 9.0701

AZD7762 A549 10.059 9.9783

AZD7762 A549 9.9783 8.9244

AZD7762 NCIH460 11.408 9.7379

AZD7762 NCIH520 8.85 8.3608

AZD7762 IGROV1 7.6


PAC-1 CCFSTTG1 14.182 13.88

PAC-1 NCIH460 14.375 14.287

PAC-1 IGROV1 13.307 12.989

PAC-1 ASPC1 13.04 12.948

PAC-1 CAL51 12.836 12.624

PAC-1 KE39 12.0 11.978

PAC-1 MKN74 13.189 12.819

PAC-1 SKUT1 13.212 11.065

SJ-172550 CCFSTTG1 15.919 14.819

SJ-172550 A549 15.089 14.443

SJ-172550 NCIH1299 15.0 14.924

SJ-172550 NCIH520 15.911 14.836

SJ-172550 OAW28 14.766 14.111

SJ-172550 ASPC1 15.061 15.0

SJ-172550 CAL51 15.553 14.821

SJ-172550 A375 14.93 13.499

SJ-172550 NCIH1869 15.237 13.155

SJ-172550 LOXIMVI 15.51 14.829

SJ-172550 DU145 15.625 15.0

SJ-172550 MKN74 15.698 14.571

SJ-172550 SKUT1 15.613 15.293

RAF265 A549 11.933 9.8961

RAF265 OAW28 12.541 11.152

RAF265 ASPC1 12.0 10.863

RAF265 SKUT1 12.837 12.109

AC55649 CCFSTTG1 14.681 14.2

AC55649 A549 14.306 14.178

AC55649 NCIH1299 14.739 14.644

AC55649 A549 14.178 13.703

AC55649 NCIH460 14.166 14.001

AC55649 IGROV1 15.0 14.606

AC55649 CAL51 15.984 14.664

AC55649 A375 14.713 14.54

AC55649 KE39 15.765 14.357

ML334 

In [5]:
compound_x_cellline

,22RV1,2313287,253J,253JBV,42MGBA,5637,639V,647V,769P,786O,...,WM983B,YAPC,YD10B,YD15,YD38,YD8,YH13,YKG1,ZR751,ZR7530
16-beta-bromoandrosterone,14.565,12.668,14.839,14.586,14.917,14.14,15.446,14.585,14.252,14.749,...,14.607,15.524,16.285,16.173,15,14.468,15.064,NaN,15.597,13.764
"1S,3R-RSL-3",9.2997,10.231,7.5951,15,6.2174,7.7638,13.301,8.0628,7.5991,5.9814,...,11.351,13.108,11.267,11.708,9.6433,5.7127,7.0035,10.699,12.43,6.2003
3-Cl-AHPC,10.259,10.418,11.077,10.379,8.9201,9.3751,10.052,9.3562,9.5917,10.551,...,11.452,12.207,9.949,9.5841,11.884,12.84,11.865,13.934,14.273,11.769
968,14.376,15.11,NaN,13.36,NaN,14.045,15.448,NaN,14.915,NaN,...,NaN,NaN,16.233,15.618,15.383,NaN,NaN,NaN,NaN,NaN
A-804598,NaN,NaN,NaN,NaN,14.147,NaN,NaN,15.312,NaN,14.498,...,14.278,15,NaN,NaN,NaN,14.548,14.678,NaN,15,14.074
AA-COCF3,12.043,9.3923,12.875,12.806,14.543,12.344,12.671,15.486,13.28,14.793,...,13.122,15.286,16.419,15,13.769,12.527,15.328,10.381,15,11.939
ABT-199,NaN,NaN,NaN,NaN,15.172,NaN,NaN,15.126,NaN,14.608,...,13.041,15.729,NaN,NaN,NaN,14.539,15.654,15.115,15,14.683
ABT-737,12.965,14.623,13.252,13.602,14.98,12.746,15.739,NaN,13.42,12.96,...,13.442,15,14.638,14.698,14.806,NaN,NaN,14.809,14.476,14.405
AC55649,13.845,13.779,14.604,14.805,NaN,13.304,15.679,15,14.751,14.746,...,14.228,15,13.788,15.787,15,14.144,14.474,14.691,15.366,14.76
AGK-2,15.266,16.757,15.176,17.642,10.679,15,14.703,15,15.958,14.633,...,13.522,15.572,13.647,14.184,15.121,14.326,16.055,16.273,15,14.676


In [ ]:
'../data/'

In [6]:
ccl

,ccl_name,snp_fp_status,culture_media,ccle_primary_site,ccle_primary_hist,ccle_hist_subtype_1,cell_line_cluster_id,master_ccl_id
index_ccl,,,,,,,,
1,CHP212,SNP-not-tested,EMEM001,autonomic_ganglia,neuroblastoma,NaN,440;513;552;589;607;628;638;647;651;657;660,144
2,IMR32,SNP-matched-reference,EMEM001,autonomic_ganglia,neuroblastoma,NaN,288;333;475;513;552;589;607;628;638;647;651;65...,475
3,KELLY,SNP-matched-reference,RPMI001,autonomic_ganglia,neuroblastoma,NaN,23;116;259;393;423;480;556;569;601;604;615;624...,528
4,KPNSI9S,SNP-not-tested,RPMI001,autonomic_ganglia,neuroblastoma,NaN,98;138;199;377;460;485;542;567;600;609;624;631...,571
5,KPNYN,SNP-matched-reference,RPMI001,autonomic_ganglia,neuroblastoma,NaN,620;652;655;657;660,572
6,MHHNB11,SNP-matched-reference,RPMI001,autonomic_ganglia,neuroblastoma,NaN,505;541;592;632;641;654;659;660,678
7,NB1,SNP-matched-reference,RPMI001,autonomic_ganglia,neuroblastoma,NaN,23;116;259;393;423;480;556;569;601;604;615;624...,720
8,NH6,SNP-matched-reference,ALPHAMEM002,autonomic_ganglia,neuroblastoma,NaN,91;333;475;513;552;589;607;628;638;647;651;657...,852
9,SIMA,SNP-matched-reference,RPMI001,autonomic_ganglia,neuroblastoma,NaN,592;632;641;654;659;660,1030
